#Creating source and destination tables

In [0]:
-- Create the source table if it does not already exist
CREATE OR REPLACE TABLE scd_cat.scd_schema.inc_source (
  id INT,                -- Unique identifier for each record
  name STRING,           -- Name field (string type)
  value INT,             -- Value field (integer type)
  updated_at TIMESTAMP,  -- Last update timestamp
  is_deleted BOOLEAN     -- Soft delete flag
);

-- Create the target table if it does not already exist
CREATE OR REPLACE TABLE scd_cat.scd_schema.inc_target (
  id INT,                -- Unique identifier for each record
  name STRING,           -- Name field (string type)
  value INT,             -- Value field (integer type)
  updated_at TIMESTAMP,  -- Last update timestamp
  is_deleted BOOLEAN     -- Soft delete flag
);

##No data in source or destination table

In [0]:
select
  *
from
  scd_cat.scd_schema.inc_source;

select
  *
from
  scd_cat.scd_schema.inc_target;

##Adding sample data in source table

In [0]:
insert into scd_cat.scd_schema.inc_source
  values 
    (1, 'a', 1, timestamp('2025-07-16 09:00:00'), false),
    (2, 'b', 2, timestamp('2025-07-16 09:05:00'), false),
    (3, 'c', 3, timestamp('2025-07-16 09:10:00'), false);

select
  *
from
  scd_cat.scd_schema.inc_source;

#Using MERGE logic for incremental load

In [0]:
MERGE INTO
  scd_cat.scd_schema.inc_target AS target
USING
  scd_cat.scd_schema.inc_source AS source
ON
  target.id = source.id
WHEN MATCHED AND
  (
    source.name <> target.name
    OR source.value <> target.value
    OR source.updated_at <> target.updated_at
    OR source.is_deleted <> target.is_deleted
  )
  THEN UPDATE SET
    target.name = source.name,
    target.value = source.value,
    target.updated_at = source.updated_at,
    target.is_deleted = source.is_deleted
WHEN NOT MATCHED THEN
  INSERT (id, name, value, updated_at, is_deleted)
  VALUES (source.id, source.name, source.value, source.updated_at, source.is_deleted);

In [0]:
select * from scd_cat.scd_schema.inc_target

In [0]:
insert into scd_cat.scd_schema.inc_source (id, name, value, updated_at, is_deleted)
  values 
    (4, 'd', 4, current_timestamp(), false),
    (5, 'e', 5, current_timestamp(), false);

update
  scd_cat.scd_schema.inc_source
set
  value = 100,
  updated_at = current_timestamp()
where
  id = 1;